<a href="https://colab.research.google.com/github/ruben216/realidadepublica/blob/main/Report_RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enviando requisições
Será necessário possuir uma chave gerada no portal de transparência. Detalhes no Readme do repositório.

In [1]:
import pandas as pd
import requests
import json
import numpy as np